In [1]:
import sys

sys.path.append("../../src/stong_core_cython")
sys.path.append("../../")

In [2]:
import numpy as np
import pandas as pd
import unrestricted_hartree_fock
import basis_data
import importlib
import concurrent.futures
from scipy.optimize import minimize

importlib.reload(unrestricted_hartree_fock)
importlib.reload(basis_data)

from atom import *
from unrestricted_hartree_fock import *
from basis_data import *

In [3]:
df = pd.read_csv("../make_base/sto_ng/sto_3g.csv")

In [4]:
def solve(zetas, atom_symbol):
    atoms = [Atom(atom_symbol, np.array([0.0, 0.0, 0.0]))]
    molecule = Molecule(atoms)
    basis_data = {atom_symbol: create_basis_data(df, zetas, 4, 1)}
    hartreefock = UnrestrictedHartreeFock(
        molecule, basis_data, max_iter=10000, tol=1e-10
    )
    hartreefock.scf()
    hartreefock.calculate_total_energy()
    results = hartreefock.get_results()
    return results, hartreefock


def get_energy(zetas, atom_symbol):
    results, _ = solve(zetas, atom_symbol)
    return results["energy"]

In [5]:
elements = [
    "H",
    "He",
    "Li",
    "Be",
    "B",
    "C",
    "N",
    "O",
    "F",
    "Ne",
    "Na",
    "Mg",
    "Al",
    "Si",
    "P",
    "S",
    "Cl",
    "Ar",
    "K",
    "Ca",
]
# elements = ["H"]
zetas = {}
results = {}
for i, element in enumerate(elements):
    print(element)
    if i == 0:
        result = minimize(
            get_energy,
            [
                19.56342956184633,
                7.74064640252246,
                3.0088773126764483,
                1.2372711236608722,
            ],
            args=(element),
        )
    else:
        result = minimize(
            get_energy,
            result.x,
            args=(element),
            bounds=[(0, None), (0, None), (0, None), (0, None)],
        )
    # result = minimize(get_energy, [2.69, 0.75, 0.3], args=(element))
    results[element] = result
    zetas[element] = result.x
    print(zetas[element])
    df_zetas = pd.DataFrame(zetas)
    df_zetas.to_csv("zetas.csv", index=False)

H
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])


LinAlgError: The leading minor of order 1 of B is not positive definite. The factorization of B could not be completed and no eigenvalues or eigenvectors were computed.

In [ ]:
results

{'H':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -0.4966449228307716
         x: [ 1.348e+00  1.182e+00]
       nit: 20
       jac: [-1.602e-07  2.235e-08]
  hess_inv: [[ 5.378e+01  1.984e+01]
             [ 1.984e+01  1.442e+01]]
      nfev: 126
      njev: 42,
 'He':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -2.839936765851108
         x: [ 2.411e+00  1.777e+00]
       nit: 11
       jac: [-5.990e-06  7.182e-06]
  hess_inv: [[ 1.745e+01  1.014e+01]
             [ 1.014e+01  8.428e+00]]
      nfev: 36
      njev: 12,
 'Li':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -7.329116402841278
         x: [ 2.692e+00  6.523e-01]
       nit: 4
       jac: [ 5.960e-06  1.788e-06]
  hess_inv: [[ 5.535e-01 -8.695e-03]
             [-8.695e-03  8.979e-01]]
      nfev: 18
      njev: 6,
 'Be':   message: Optimization terminated successfully.
   success: 